In [1]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from numpy.fft import fft, fftfreq
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers

In [2]:
train_data = np.load('./train_data.npy')
val_data = np.load('./val_data.npy')
test_data = np.load('./test_data.npy')

train_labels = np.load('./train_labels.npy')
val_labels = np.load('./val_labels.npy')
test_labels = np.load('./test_labels.npy')

In [3]:
np.savez_compressed("compressed_data.npz", train_data)

### 学習

In [5]:
class CRNN(keras.Model):
  def __init__(self,num_classes=5,sequence_size=16,img_size=180):
    super().__init__()
    #self.inputs = keras.Input(shape=(img_size,img_size,3))
    self.num_classes = num_classes
    self.sequence_size = sequence_size
    self.img_size = img_size
    self.rescale = layers.Rescaling(1./255)
    self.conv2d_1 = layers.Conv2D(filters=32,kernel_size=3,activation="relu")
    self.drop = layers.Dropout(0.8)
    self.max_1 = layers.MaxPooling2D(pool_size=2)
    self.conv2d_2 = layers.Conv2D(filters=64,kernel_size=3,activation="relu")
    self.max_2 = layers.MaxPooling2D(pool_size=2)
    #self.conv2d_3 = layers.Conv2D(filters=128,kernel_size=3,activation="relu")
    #self.max_3 = layers.MaxPooling2D(pool_size=2)
    #self.conv_4 = layers.Conv2D(filters=256,kernel_size=3,activation="relu")
    #self.max_4 = layers.MaxPooling2D(pool_size=2)
    self.flatten = layers.Flatten()
    #self.dense = layers.Dense(256,activation="relu")
    #self.out = layers.Dense(self.num_classes,activation="softmax")

  def call(self,inputs):
    x = self.rescale(inputs)
    x = self.conv2d_1(x)
    x = self.max_1(x)
    x = self.conv2d_2(x)
    x = self.max_2(x)
    #x = self.conv2d_3(x)
    #x = self.max_3(x)
    #x = self.conv_4(x)
    #x = self.max_4(x)
    #x = self.flatten(x)
    x = self.flatten(x)
    #x = self.dense(x)
    #return  self.out(x)
    return self.drop(x)
    #mdoel = keras.Model(inputs=inputs,outputs=outputs)
    #return model
    #model = keras.Model(inputs=inputs,outputs=outputs)
    #return model
  def get_config(self):
    return {}




output_list = []
inputs = keras.Input(shape=(16,90,90,3))
inputs_i = tf.transpose(inputs,perm=[1,0,2,3,4])
for i in range(16):
  x = CRNN()(inputs_i[i])
  output_list.append(x)
combined = tf.stack(output_list)
x = tf.transpose(combined,perm=[1,0,2])
x = layers.SimpleRNN(128,return_sequences=True)(x)
#x = layers.LSTM(128,return_sequences=True)(x)


#x = layers.Dropout(0.5)(x)
outputs = layers.Dense(7,activation="softmax")(x)
model = keras.Model(inputs=inputs,outputs=outputs)
#model.compile(loss="categorical_crossentropy",optimizer="rmsprop",metrics=["accuracy"])
model.compile(loss="categorical_crossentropy",optimizer=tf.optimizers.RMSprop())

In [8]:
#history = model.fit(train_data,train_labels,epochs=1,batch_size=1,
#                    validation_data=(val_data,val_labels)
#                   )

1312/1312 [==============================] - 158s 118ms/step - loss: 1.3229 - val_loss: 1.5754


In [49]:
#model.save('./model.keras')
#model.save_weights('./model.keras')

In [12]:
test_model = keras.models.load_model('model.keras',custom_objects={"CRNN": CRNN})

In [14]:
#model.load_weights('weights.keras')

### 推論

In [13]:
def make_text(data):
  #text=""
  p = test_model.predict(data)
  text = ""
  for i in range(data.shape[0]):
    #for j in range(16):
    text += str(np.argmax(p[i][3]))
    if i % 16 == 15:
        print(text + ",")
        text = ""
    #print(text+',')
    #text += "\n"
  return text

In [14]:
text = make_text(test_data)

32/32 [==============================] - 6s 170ms/step
0000000000000122,
2220000000202202,
2220200002021022,
1021102010121020,
1022122110201000,
1021102110211000,
1011201210210101,
1020102120111011,
1010111011101101,
0222102212202001,
2011102110221110,
1020102000201020,
0000100010222011,
0022102120211020,
1011100120111102,
0122101120111011,
1011101010201111,
2001102210001020,
1010111110211110,
0112100000001001,
1022102020221111,
1011110110111210,
1020100111101211,
1001101112001111,
1011110020010100,
2000102010201011,
1020202012201112,
1211120010011022,
1222000220211111,
2000102010101000,
2000102011211022,
1120102011021011,
1000020000001011,
2020102000021010,
0002020122221000,
1011000010021221,
1221102210212110,
2010202010100000,
0020201120211001,
1120100220221010,
2111102010111021,
0011000020010000,
1000200210101000,
2000100000001010,
1020212111102001,
1020102110221002,
1011201220102011,
1021100120211111,
1011101010211110,
1022102210201002,
2021101110211120,
1120102200011010,
202010101